## DeepCut2RealTime
This is a notebook-based implementation of our real-time behaviour estimation and
reinforcement system. It is adapted from the code that was used to carry out the behavioural experiments outlined
in our _eNeuro_ paper: [Forys, Xiao, Gupta, and Murphy (2020)](https://doi.org/10.1523/ENEURO.0096-20.2020),
and builds upon the code outlined in our _bioRxiV_ preprint: [Forys, Xiao, Gupta, Boyd, and Murphy (2018)](https://doi.org/10.1101/482349). It's based on the [DeepLabCut toolbox](http://www.mousemotorlab.org/deeplabcut) created by [Mathis et al. (2018)](https://doi.org/10.1038/s41593-018-0209-y).

This notebook will allow you to track user-defined behaviours in real time from your webcam.
In order to do this, we'll:
1. Set up DeepLabCut and our custom real-time tracking code
2. Select or upload a DeepLabCut model to define the behaviour to be tracked
3. Define our criterion behaviour
4. Start tracking the behaviour!

NOTE: for best results, ensure that you are using a GPU by going to the menu above and clicking `Runtime > Change runtime type`, then selecting `GPU` from the dropdown menu. 

If you're viewing this page in GitHub, click the following button to run this code in Google Colab: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bf777/deepcut2realtime/blob/master/notebooks/deepcut2realtime.ipynb)

In [ ]:
# To begin, let's install some necessary modules (this step
# may take a few minutes).
!pip install deeplabcut pandas scikit-image Pillow opencv-python
%reload_ext numpy

In [ ]:
# Use TensorFlow 1.x:
%tensorflow_version 1.x

In [ ]:
#GUIs don't work on the cloud, so we supress them:
import os
os.environ["DLClight"]="True"

# stifle tensorflow warnings, like we get it already.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Now, let's import the modules we need.
import os.path
from deeplabcut.pose_estimation_tensorflow.nnet import predict
from deeplabcut.pose_estimation_tensorflow.config import load_config
from deeplabcut.utils import auxiliaryfunctions
import time
import pandas as pd
import numpy as np
import os
import cv2
from PIL import Image
from io import BytesIO
import base64
from pathlib import Path
import tensorflow as tf
import skimage
from skimage.util import img_as_ubyte
import logging

# Real-time tracking dependencies
import _thread
from copy import deepcopy

In [ ]:
# Mount google drive. If you save your custom DeepLabCut model in your Google Drive,
# you'll be able to access it here. Additionally, this will let you directly save
# the outputs of your model to Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Now, we'll define `analyze_stream`, the top-level function for running the real-time tracking
(adapted from DeepLabCut):

We will define a criterion movement for the body part that we're analyzing. If the body part moves in a way that
satisfies this criterion between two frames, we can record the behaviour as having met criterion (demonstrating how
feedback would be given based on this user-defined movement).

In [6]:
# THRESHOLDS FOR Y MOVEMENT
# min y movement on left paw to be counted as a trigger = 5 px
y_left = 5 #@param {type: "number"}
# max y movement on right paw to be counted as a trigger = 10 px
y_right = 10 #@param {type: "number"}
# Max y movement overall: 100 px
y_upper_lim = 100 #@param {type: "number"}
# Refractory time (s): time after last trigger during which no LED flash can be triggered/reward given
refractory_time = 0.3 #@param {type: "number"}

In [7]:
def analyze_stream(config, destfolder, y_left, y_right, y_upper_lim, refractory_time,
                   shuffle=1, trainingsetindex=0, gputouse=0, save_as_csv=False, save_frames=True,
                   cropping=None, baseline=True, name="default_animal", camtype="colab"):
    if 'TF_CUDNN_USE_AUTOTUNE' in os.environ:
        del os.environ['TF_CUDNN_USE_AUTOTUNE']  # was potentially set during training

    if gputouse is not None:  # gpu selection
        os.environ['CUDA_VISIBLE_DEVICES'] = str(gputouse)

    tf.reset_default_graph()
    start_path = os.getcwd()  # record cwd to return to this directory in the end

    cfg = auxiliaryfunctions.read_config(config)

    if cropping is not None:
        cfg['cropping'] = True
        cfg['x1'], cfg['x2'], cfg['y1'], cfg['y2'] = cropping
        print("Overwriting cropping parameters:", cropping)
        print("These are used for all videos, but won't be save to the cfg file.")

    trainFraction = cfg['TrainingFraction'][trainingsetindex]

    modelfolder = os.path.join(cfg["project_path"], str(auxiliaryfunctions.GetModelFolder(trainFraction, shuffle, cfg)))
    path_test_config = Path(modelfolder) / 'test' / 'pose_cfg.yaml'
    try:
        dlc_cfg = load_config(str(path_test_config))
    except FileNotFoundError:
        raise FileNotFoundError(
            "It seems the model for shuffle %s and trainFraction %s does not exist." % (shuffle, trainFraction))

    # Check which snapshots are available and sort them by # iterations
    try:
        Snapshots = np.array(
            [fn.split('.')[0] for fn in os.listdir(os.path.join(modelfolder, 'train')) if "index" in fn])
    except FileNotFoundError:
        raise FileNotFoundError(
            "Snapshots not found! It seems the dataset for shuffle %s has not been trained/does not exist.\n Please train it before using it to analyze videos.\n Use the function 'train_network' to train the network for shuffle %s." % (
            shuffle, shuffle))

    if cfg['snapshotindex'] == 'all':
        print(
            "Snapshotindex is set to 'all' in the config.yaml file. Running video analysis with all snapshots is very costly! Use the function 'evaluate_network' to choose the best the snapshot. For now, changing snapshot index to -1!")
        snapshotindex = -1
    else:
        snapshotindex = cfg['snapshotindex']

    increasing_indices = np.argsort([int(m.split('-')[1]) for m in Snapshots])
    Snapshots = Snapshots[increasing_indices]

    print("Using %s" % Snapshots[snapshotindex], "for model", modelfolder)

    ##################################################
    # Load and setup CNN part detector
    ##################################################

    # Check if data already was generated:
    dlc_cfg['init_weights'] = os.path.join(modelfolder, 'train', Snapshots[snapshotindex])
    trainingsiterations = (dlc_cfg['init_weights'].split(os.sep)[-1]).split('-')[-1]

    # update batchsize (based on parameters in config.yaml)
    dlc_cfg['batch_size'] = cfg['batch_size']
    # Name for scorer:
    DLCscorer = auxiliaryfunctions.GetScorerName(cfg, shuffle, trainFraction, trainingsiterations=trainingsiterations)

    sess, inputs, outputs = predict.setup_pose_prediction(dlc_cfg)
    pdindex = pd.MultiIndex.from_product([[DLCscorer], dlc_cfg['all_joints_names'], ['x', 'y', 'likelihood']],
                                         names=['scorer', 'bodyparts', 'coords'])

    if gputouse is not None:  # gpu selectinon
        os.environ['CUDA_VISIBLE_DEVICES'] = str(gputouse)

    ##################################################
    # Set up data buffer and global variables to be used in threads
    ##################################################
    global PredicteData
    PredicteData = np.zeros((50000, 3 * len(dlc_cfg['all_joints_names'])))
    global led_arr
    led_arr = np.zeros((50000, 7))
    global x_range
    global y_range
    global acc_range
    x_range = list(range(0, (3 * len(dlc_cfg['all_joints_names'])), 3))
    y_range = list(range(1, (3 * len(dlc_cfg['all_joints_names'])), 3))
    acc_range = list(range(2, (3 * len(dlc_cfg['all_joints_names'])), 3))
    global colors
    colors = [(0, 0, 255), (0, 165, 255), (0, 255, 255), (0, 255, 0), (255, 0, 0), (240, 32, 160), (0, 0, 255),
              (0, 165, 255)]
    global empty_count
    empty_count = 0
    global threshold_count
    AnalyzeStream(DLCscorer, trainFraction, cfg, dlc_cfg, sess, inputs, outputs, pdindex, save_as_csv, save_frames,
                  destfolder, name, baseline, camtype, y_left, y_right, y_upper_lim, refractory_time)

The next function we'll define is `GetPoseS`, our modified version of
the DeepLabCut function of the same name that takes a batch of two
frames from the webcam as an input and sets up pose estimation.

In [8]:
from google.colab import output

def GetPoseS(cfg, dlc_cfg, sess, inputs, outputs, cap, w, h, nframes, save_frames, destfolder, baseline, camtype,
             y_left, y_right, y_upper_lim, refractory_time):
    """ Non batch wise pose estimation for video cap."""
    # Prepare data arrays
    global lastFrameWasMoved
    global thisFrameWasMoved
    global threshold_count
    lastFrameWasMoved = False
    thisFrameWasMoved = False
    x_arr = []
    y_arr = []
    if cfg['cropping']:
        print(
            "Cropping based on the x1 = %s x2 = %s y1 = %s y2 = %s. You can adjust the cropping coordinates in the config.yaml file." % (
            cfg['x1'], cfg['x2'], cfg['y1'], cfg['y2']))
        if w > 0 and h > 0:
            pass
        else:
            raise Exception('Please check the order of cropping parameter!')
    LED = ''
    start = time.time()
    counter = 0
    threshold_count = 0
    frame_arr = []
    try:
        while cap:
            if camtype == 'cv2':
                ret, frame = cap.read()
            elif camtype == 'colab':
                frame = take_photo()
            # if ret:
            if len(frame) > 0:
                # print("FRAME RECEIVED")
                frame = skimage.color.gray2rgb(frame)
                if cfg['cropping']:
                    frame = img_as_ubyte(frame[cfg['y1']:cfg['y2'], cfg['x1']:cfg['x2']])
                else:
                    frame = img_as_ubyte(frame)
                frame_arr.append(frame)
                frame_time = time.time()
                led_arr[counter, 0] = frame_time
                led_arr[counter + 1, 0] = frame_time
                if (time.time() - start) > 0:
                    print("Current FPS: {} fps, Time elapsed: {} s, Number of frames so far: {}".format(
                        round(counter / (time.time() - start), 2),
                        round(time.time() - start, 2), counter))
                if len(frame_arr) == 2:
                    # This thread carries out pose estimation on each batch of two frames that arrives from the camera.
                    _thread.start_new_thread(frame_process, (frame_arr, dlc_cfg, sess, inputs, outputs, counter,
                                                             save_frames, destfolder, LED, x_arr, y_arr, frame_time,
                                                             start, baseline, y_left, y_right, y_upper_lim,
                                                             refractory_time))
                    frame_arr = []
                    counter += 2
            x_arr = []
            y_arr = []
            x_first = 0
            y_first = 0
            threshold = 0
            # Run each trial for 130 seconds
            if time.time() - start >= 30:
                nframes = counter
                break
    except KeyboardInterrupt:
        _thread.exit()
        print("Finished.")
        cap.release()
        exit()
    # LED.close()
    return nframes

Now, we'll define `AnalyzeStream`, which manages the overall
operation of the real-time tracking trial and which handles the camera
connection and data saving.

In [9]:
def AnalyzeStream(DLCscorer, trainFraction, cfg, dlc_cfg, sess, inputs, outputs, pdindex, save_as_csv, save_frames,
                  destfolder, name, baseline, camtype, y_left, y_right, y_upper_lim, refractory_time):
    """Sets up camera connection for pose estimation, and handles data output."""
    # Setup camera connection
    # if camtype == 'cv2':
    #     cam = cv2.VideoCapture(0)
    #     print("CV2-compatible camera connected!")
    #     size = (int(cam.get(3)), int(cam.get(4)))
    if camtype == 'colab':
        cam = 'colab'

    print("Starting to analyze stream")
    # Accept a single connection and make a file-like object out of it
    cap = cam
    dataname = os.path.join(destfolder, '{}_{}.h5'.format(DLCscorer, name))
    dataname_led = os.path.join(destfolder, '{}_{}_LED.h5'.format(DLCscorer, name))
    led_data_cols = ['FrameTime', 'MovementDiffLeft', 'MovementDiffRight', 'ThresholdTime', 'Delay', 'FlashTime',
                     'WaterTime']
    size = (200, 200)
    w, h = size
    shutter = 1/500
    brightness = 68
    v_blanking = 982
    acc_tolerance = 0.20
    missing_count = 0
    nframes = 0

    print("Starting to extract posture")
    start = time.time()
    nframes = GetPoseS(cfg, dlc_cfg, sess, inputs, outputs, cap, w, h, nframes, save_frames, destfolder, baseline,
                       camtype, y_left, y_right, y_upper_lim, refractory_time)

    # stop the timer and display FPS information
    stop = time.time()
    fps = nframes / (stop - start)
    print("\n")
    print("[INFO] elasped time: {:.2f}".format(stop - start))
    print("[INFO] approx. FPS: {:.2f}".format(fps))

    # If there's rows with blank data at the end of the trial, record this data as missing/dropped frames
    for row in PredicteData[int(np.around(fps * 10)):nframes, :]:
        if 0 in row:
            missing_count += 1

    time.sleep(10)
    avg_array = led_arr[:, 4]
    avg_delay = avg_array[avg_array != 0].mean()
    sd_delay = np.std(avg_array[avg_array != 0])
    avg_acc = PredicteData[:nframes, acc_range].mean()

    # Prints out results of trial
    print("Empty values: {}, {} per second".format(str(missing_count), str(missing_count / (stop - start))))
    print("Adjusted frame rate: {}".format(str((nframes - missing_count) / (stop - start))))
    print("Average delay: {} s".format(str(avg_delay)))
    print("Standard dev. of delay: {} s".format((str(sd_delay))))
    print("Average tracking accuracy: {}".format((str(avg_acc))))

    # Save metadata with trial information and camera information
    dictionary = {
        "name": name,
        "start": start,
        "stop": stop,
        "run_duration": stop - start,
        "Scorer": DLCscorer,
        "DLC-model-config file": dlc_cfg,
        "fps": fps,
        "fps_adjusted": ((nframes - missing_count) / (stop - start)),
        "avg_delay": avg_delay,
        "sd_delay": sd_delay,
        "v_blanking": v_blanking,
        "shutter": shutter,
        "brightness": brightness,
        "batch_size": dlc_cfg["batch_size"],
        "frame_dimensions": (h, w),
        "nframes": nframes,
        "acc_tolerance": acc_tolerance,
        "avg_acc": avg_acc,
        "iteration (active-learning)": cfg["iteration"],
        "training set fraction": trainFraction,
        "cropping": cfg['cropping'],
        "LED_time": 0.2,
        "water_time": 0.15,
        "refractory_period": 0.3
    }
    metadata = {'data': dictionary}

    print("Saving results in {} and {}".format(dataname, dataname_led))
    auxiliaryfunctions.SaveData(PredicteData[:nframes, :], metadata, dataname, pdindex, range(nframes), save_as_csv)
    auxiliaryfunctions.SaveData(led_arr[:nframes, :], metadata, dataname_led, led_data_cols, range(nframes),
                                save_as_csv)
    # cam.release()

The function `frame_process` handles the pose estimation for each batch
of two frames.

In [10]:
def frame_process(frame_arr, dlc_cfg, sess, inputs, outputs, counter, save_frames, destfolder, LED, x_arr, y_arr,
                  frame_time, start, baseline, y_left, y_right, y_upper_lim, refractory_time):
    """Estimates pose in each frame, and optionally plots the pose on each frame."""
    # Set up parameters for refractory period (to check whether there was movement on the previous frame).
    global thisFrameWasMoved
    global lastFrameWasMoved
    global threshold_time
    global threshold_count
    x_avg_left = []
    y_avg_left = []
    x_avg_right = []
    y_avg_right = []
    # Run DeepLabCut pose prediction on each batch of two frames
    for n, frame in enumerate(frame_arr):
        pose = predict.getpose(frame, dlc_cfg, sess, inputs, outputs)
        # if time.time() - start >= 10:
        PredicteData[counter + n, :] = pose.flatten()  # NOTE: thereby cfg['all_joints_names'] should be same order as bodyparts!
        # Create lists to store average x and y paw positions
        for x_val, y_val in zip(x_range, y_range):
            x_arr.append(PredicteData[counter + n, :][x_val])
            y_arr.append(PredicteData[counter + n, :][y_val])
        if n == 0:
            add = 0
        elif n == 1:
            add = 8
        x_avg_right.append(np.mean(x_arr[0 + add:3 + add]))
        x_avg_left.append(np.mean(x_arr[4 + add:7 + add]))
        y_avg_right.append(np.mean(y_arr[0 + add:3 + add]))
        y_avg_left.append(np.mean(y_arr[4 + add:7 + add]))
        # print(y_avg_left)
        # print(y_avg_right)
        thisFrameWasMoved = False
        # print("POSE ESTIMATION LENGTH: {}".format(n))
        # print("LENGTH: {}".format(len(y_avg_left)))
        if len(y_avg_left) >= 2:
          threshold_calc(counter, start, y_avg_left, y_avg_right, y_upper_lim, 
                         thisFrameWasMoved, y_left, y_right, refractory_time, 
                         frame_time, threshold_count, lastFrameWasMoved)
        if save_frames:
          _thread.start_new_thread(frame_save_func, (frame, x_range, y_range, 
                                                     x_avg_left, y_avg_left,
                                                      x_avg_right, y_avg_right, 
                                                     destfolder, n, counter))

Now, we'll define the function used to evaluate whether movement between these two frames was greater than the criterion
that we defined. If it is, we'll record that the movement was beyond criterion.

In [11]:
def threshold_calc(counter, start, y_avg_left, y_avg_right, y_upper_lim, 
                   thisFrameWasMoved, y_left, y_right, refractory_time, 
                   frame_time, threshold_count, lastFrameWasMoved):
    # We wait for 10.1 s after the start of the trial to start giving behavioural feedback. This is because the latency
    # for pose estimation is longer, and the framerate is less stable, in the first 10 s of the trial
    # (likely because of Python library setup).
    if counter >= 1:
        # print("Left: {}".format(y_avg_left))
        # print("Right: {}".format(y_avg_right))
        trial_start = True
        # if not baseline:
        #     _thread.start_new_thread(led_task, (LED, trial_start, trial_length - 10)),
        led_arr[counter, 1] = abs(y_avg_left[1] - y_avg_left[0])
        led_arr[counter, 2] = abs(y_avg_right[1] - y_avg_right[0])

        '''
        Below is the code used to define the criterion for movement. By default,
        it's configured for eight-point tracking in which the first four points
        are the right paw and the last four points are the left paw. If the left
        paw moves more than a given criterion, the movement will be recorded as
        such in the .csv output file labelled "LED". In our actual code, this
        would be used to trigger an LED flash and water reward (handled by the
        commented-out `thresholdFunc` function). Feel free to play around with
        this code, along with adjusting the parameters as defined in an earlier
        cell.
        '''
        if (y_left <= abs(y_avg_left[1] - y_avg_left[0]) <= y_upper_lim and
                y_right >= abs(y_avg_right[1] - y_avg_right[0])):
            # and PredicteData[counter, acc_range].mean() >= 0.20
            print("CRITERION MOVEMENT: {}".format(abs(y_avg_left[1] - 
                                                      y_avg_left[0])))
            thisFrameWasMoved = True
            if threshold_count == 0:
                threshold_time = time.time() - 0.5
        if thisFrameWasMoved and not lastFrameWasMoved and abs(time.time() - threshold_time) >= refractory_time:
            threshold_time = time.time()
            threshold_count += 1
            delay = threshold_time - frame_time
            led_arr[counter + 1, 3] = threshold_time
            led_arr[counter + 1, 4] = delay
            ttt = True
            # _thread.start_new_thread(thresholdFunc, (LED, baseline, ttt, gpio_light_time, gpio_water_time, counter,
            #                                          threshold_time))
        lastFrameWasMoved = thisFrameWasMoved

In [12]:
# Save frames
def frame_save_func(frame, x_range, y_range, x_avg_left, y_avg_left, 
                    x_avg_right, y_avg_right, destfolder, n, counter):
    '''Handles plotting estimated paw positions on frames, and saving these frames.'''
    tmp = deepcopy(frame)
    avgs = False
    if avgs:
        cv2.circle(tmp, (int(x_avg_right[n]), int(y_avg_right[n])), 6, 
                   (0, 165, 255), -1)
        cv2.circle(tmp, (int(x_avg_left[n]), int(y_avg_left[n])), 6, 
                   (0, 165, 255), -1)
    else:
        for x_plt, y_plt, c in zip(x_range, y_range, colors):
            cv2.circle(tmp, (int(PredicteData[counter + n, :][x_plt]),
                             int(PredicteData[counter + n, :][y_plt])), 2, c, -1)
    cv2.imwrite(os.path.join(destfolder, 
                             'frame{}.png'.format(str(counter + n))), tmp)

In [21]:
'''
Set folder for data input/output in your Google drive (make sure to
create a folder with this name in your Google drive!)
'''
gDriveParentFolder = 'deepcut2realtime' #@param {type: "string"}

We have now defined all of the code that we need to run the real-time tracking and record data accordingly.
Now we need to select the behavioural model to use, and connect to the webcam.

To pick a pretrained model from the [DeepLabCut Model Zoo](http://www.mousemotorlab.org/dlc-modelzoo/),
run the following two cells. To upload your own model, skip the next two cells and run the one that follows them. The
following two cells are from the DeepLabCut developers' Model Zoo [Colab notebook](https://colab.research.google.com/github/AlexEMG/DeepLabCut/blob/master/examples/COLAB_DLC_ModelZoo.ipynb).

In [13]:
# Select a DLC ModelZoo model
import deeplabcut
import ipywidgets as widgets
from IPython.display import display
model_options = deeplabcut.create_project.modelzoo.Modeloptions
model_selection = widgets.Dropdown(
    options=model_options,
    value=model_options[0],
    description="Choose a DLC ModelZoo model!",
    disabled=False
)
display(model_selection)
modelzoo = True

Dropdown(description='Choose a DLC ModelZoo model!', options=('full_human', 'full_cat', 'full_dog', 'primate_f…

In [14]:
# You can change the name of the project here by editing ProjectFolderName and YourName
ProjectFolderName = 'myDLC_modelZoo' #@param {type: "string"}
YourName = 'teamDLC' #@param {type: "string"}
model2use = model_selection.value # see 
# http://www.mousemotorlab.org/dlc-modelzoo for the list! (curently: full_dog,
# full_cat, full_human, primate_face)

In [ ]:
'''
ONLY run this cell if you want to upload your own model! If so, upload your
project folder to google drive, then add the name of the folder where you're 
keeping all of your outputs under `gDriveParentFolder` and your DeepLabCut 
folder here.
'''
ManualProjectFolderName = 'latest_model' #@param {type: "string"}
# Edit the config file's project path to work on Colab.
config_path = os.path.join('/content/drive/My Drive/', gDriveParentFolder, 
                           ManualProjectFolderName, 'config.yaml')
config_path_parent = os.path.join('/content/drive/My Drive/', gDriveParentFolder, 
                           ManualProjectFolderName)
edits = {'project_path': config_path_parent}
auxiliaryfunctions.edit_config(config_path, edits)
modelzoo = False

In [ ]:
config_path

If we've selected a model from the DeepLabCut Model Zoo, we now need to generate a config folder for the project. Don't
run the following code cell if you're using your own model!

In [ ]:
print(os.getcwd())
'''
DeepLabCut expects a video - even though we don't need one for real-time
tracking ;) - so we've provided a placeholder video in the git repository that
will be used to keep DeepLabCut happy.
'''
# Fetch a single <1MB file using the raw GitHub URL.
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://api.github.com/repos/bf777/DeepCut2RealTime/contents/notebooks/test.avi
video_path = '.'
videotype = '.avi'
config_path = deeplabcut.create_pretrained_project(ProjectFolderName, YourName, [video_path],
                                                        videotype=videotype, model=model2use, analyzevideo=False,
                                                        createlabeledvideo=False, copy_videos=False)

Now, we will handle the video stream from your webcam through JavaScript. Running the following two cells will
define the functions necessary to establish the stream.

In [18]:
'''
ADAPTED FROM https://emadehsan.com/p/object-detection,
https://colab.research.google.com/drive/1HPrxuPjJDvEx64TlY7K8SPK_XbOJTZ7A
## Camera Capture
Using a webcam to capture images for processing on the runtime.
Source: https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb#scrollTo=2viqYx97hPMi
'''

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.5):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const video = document.createElement('video');
      video.style.display = 'none';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      // show the video in the HTML element
      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      // google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // prints the logs to cell
      let jsLog = function(abc) {
        document.querySelector("#output-area").appendChild(document.createTextNode(`${abc}... `));
      }

      // Wait for Capture to be clicked.
      // await new Promise((resolve) => capture.onclick = resolve);
        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;

      for (let i = 0; i < 1; i++) {
        canvas.style.display = "none";
        canvas.getContext('2d').drawImage(video, 0, 0);
        img = canvas.toDataURL('image/jpeg', quality);

        // show each captured image
        // let imgTag = document.createElement('img');
        // imgTag.src = img;
        // div.appendChild(imgTag);

        // jsLog(i + "sending")
        // Call a python function and send this image
        // google.colab.kernel.invokeFunction('notebook.get_img', [img], {});
        // jsLog(i + "SENT")

        // wait for X miliseconds second, before next capture
        // await new Promise(resolve => setTimeout(resolve, 10));
      }
      stream.getVideoTracks()[0].stop(); // stop video stream
      return(img)
    }
    ''')
  display(js) # make the provided HTML, part of the cell
  data = eval_js('takePhoto({})'.format(quality)) # call the takePhoto() JavaScript function
  output_frame = get_img(data)
  return output_frame 

In [19]:
def data_uri_to_img(uri):
  """convert base64image to numpy array"""
  try:
    image = base64.b64decode(uri.split(',')[1], validate=True)
    # make the binary image, a PIL image
    image = Image.open(BytesIO(image))
    # convert to numpy array
    image = np.array(image, dtype=np.uint8)
    return image
  except Exception as e:
    logging.exception(e);print('\n')
    return None

def get_img(img):
    frame = data_uri_to_img(img)
    frame = skimage.color.gray2rgb(frame)
    return frame

At last, we are ready to run the script!

NOTE: Currently this cloud-based implementation runs very slowly (<1 FPS)
and is intended as a demonstration of the code's structure only. For best results, follow the instructions [on our GitHub](https://github.com/bf777/DeepCut2RealTime) to run our software locally.

In [23]:
# Change batch size to 1
edits = {'batch_size': 1}
if modelzoo:
  auxiliaryfunctions.edit_config(config_path[0], edits)
else:
  auxiliaryfunctions.edit_config(config_path, edits)

In [22]:
# Output folder (for saving data and images)
destfolder = os.path.join('/content/drive/My Drive', gDriveParentFolder)

In [ ]:
'''
Run the script itself!
NOTE: only set save_frames=True if you have sufficient space in your google
drive!! This option saves ALL frames that are processed if set to True.
'''
if modelzoo:
  analyze_stream(config_path[0], destfolder, y_left, y_right, y_upper_lim, 
               refractory_time, save_frames=False, save_as_csv=True)
else:
  analyze_stream(config_path, destfolder, y_left, y_right, y_upper_lim, 
               refractory_time, save_frames=False, save_as_csv=True)

# Docstring for analyze_stream:



```
# Docstring for analyze_stream:
"""
def analyze_stream(config, destfolder, shuffle=1, trainingsetindex=0, gputouse=0, save_as_csv=False, save_frames=True,
                   cropping=None, baseline=True, name="default_animal", camtype="colab"):
Makes prediction based on a trained network. The index of the trained network is specified by parameters in the config file (in particular the variable 'snapshotindex')

You can crop the video (before analysis), by changing 'cropping'=True and setting 'x1','x2','y1','y2' in the config file. The same cropping parameters will then be used for creating the video.

Output: The labels are stored as MultiIndex Pandas Array, which contains the name of the network, body part name, (x, y) label position \n
        in pixels, and the likelihood for each frame per body part. These arrays are stored in an efficient Hierarchical Data Format (HDF) \n
        in the same directory, where the video is stored. However, if the flag save_as_csv is set to True, the data can also be exported in \n
        comma-separated values format (.csv), which in turn can be imported in many programs, such as MATLAB, R, Prism, etc.

Parameters
----------
config : string
    Full path of the config.yaml file as a string.
destfolder : string
    Full path of the directory to which you want to output data and (optionally) saved frames.

shuffle: int, optional
    An integer specifying the shuffle index of the training dataset used for training the network. The default is 1.
trainingsetindex: int, optional
    Integer specifying which TrainingsetFraction to use. By default the first (note that TrainingFraction is a list in config.yaml).

gputouse: int, optional. Natural number indicating the number of your GPU (see number in nvidia-smi). If you do not have a GPU put None.
See: https://nvidia.custhelp.com/app/answers/detail/a_id/3751/~/useful-nvidia-smi-queries
save_as_csv: bool, optional
    Saves the predictions in a .csv file. The default is ``False``; if provided it must be either ``True`` or ``False``
save_frames: bool, optional
    Labels and saves each frame of the stream to the destfolder defined above. The default is ``False``; if provided it must be either ``True`` or ``False``
cropping: bool, optional
    Selects whether to apply cropping to each frame or not. Not recommended as it increases computational overhead.
baseline: bool, optional
    Selects whether the current trial is a baseline trial (movement tracking but no reinforcement) or a training trial (movement tracking with reinforcement
    via water reward). If True, current trial is baseline trial; else (e.g. False), current trial is training trial.
name: string, optional
    Pass in the name/subject ID of the animal to be observed in the current trial. This will ensure that the animal is named consistently in data output and
    trial metadata.
camtype: string, optional
    Pass in the camera configuration to use. If using a sentech camera, use `sentech`. If using a standard OpenCV webcam
    when running the script on your local machine (not Colab!), use `opencv`. In Colab, use `colab` (will access webcam via Javascript).
Examples
--------
If you want to analyze a stream without saving anything
>>> deeplabcut.analyze_stream('/analysis/project/reaching-task/config.yaml','/analysis/project/reaching-task/output')
--------
If you want to analyze a stream and save just the labelled frames
>>> deeplabcut.analyze_stream('/analysis/project/reaching-task/config.yaml','/analysis/project/reaching-task/output', save_frames=True)
--------
If you want to analyze a stream and save both the frames and a .csv file with the coordinates of the labels
>>> deeplabcut.analyze_stream('/analysis/project/reaching-task/config.yaml','/analysis/project/reaching-task/output', save_as_csv=True, save_frames=True)
--------
"""
```

